# 0: DataDataData

Here's a sample class wrapping the main abstraction, the `Dataset`.

* TODO Support for multiple users? @ben

In [1]:
from dataset import build_sample_corpus, Dataset
train = build_sample_corpus()

# 1.1 Annotation & reliability

* TODO Learning curve @ben
* TODO Agreement @ben


## Samplers

For example, a random unlabelled sampler.

## Manually label some examples

In [2]:
from annotator import AnnotationPane
from samplers import Random
pane = AnnotationPane(train, Random(10))

VBox(children=(HBox(children=(Button(description='Yes', style=ButtonStyle()), Button(description='No', style=ButtonStyle()))), HTML(value="<p>From: yongje@hardy.u.washington.edu (Yong Je Lim)\nSubject: Dealer cheated me with wrong odometer reading. Need help!\nOrganization: University of Washington, Seattle\nLines: 14\nDistribution: usa\nNNTP-Posting-Host: hardy.u.washington.edu\n\nHere is a story.  I bought a car about two weeks ago.  I finally can\nget hold of the previous owner of the car and got all maintanence\nhistory of the car.  In between '91 and '92, the instrument pannel \nof the car has been replaced and the odometer also has been reset\nto zero.  Therefore, the true meter reading is the reading before\nreplacement plus current mileage.  That shows 35000 mile difference\ncomparing to the mileage on the odometer disclosure from.  The \ndealer never told me anything about that important story.\n\nI hope that I can return the car with full refund.  Do u think this\nis possible?  Does anyone have similar experiences?  Any comments\nwill be appreciated.  Thanks.\n\nyongje@u.washington.edu \n</p>")))

## See our new labels in the dataset

In [3]:
print(train.label_distribution)

{None: 11197, False: 113, True: 4}


# 1.1: Active learning

Here is a straw man active sampler that:
* trains a classifier on the labelled data
* predicts the labels of unlabelled data
* selects text with a specific label profile

In [4]:
from classifier import train_classifier
from samplers import Active

labelled = ((text, label) for text, label in train if label in {True, False})
pipeline = train_classifier(labelled, cv=3)
pane = AnnotationPane(train, Active(pipeline, 
                                    accept=lambda pred: pred[True] > 0.8,
                                    limit=10))

Got 117 labelled samples
Cross-validating
Cross-validated accuracy: 0.95 (+/- 0.04)
Refitting


/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will b

Predicting 11197 unlabelled


/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


VBox(children=(HBox(children=(Button(description='Yes', style=ButtonStyle()), Button(description='No', style=ButtonStyle()))), HTML(value='<p>From: roby@chopin.udel.edu (Scott W Roby)\nSubject: Re: BATF/FBI Murders Almost Everyone in Waco Today! 4/19\nNntp-Posting-Host: chopin.udel.edu\nOrganization: University of Delaware\nLines: 45\n\nIn article <1993Apr20.195636.17742@guinness.idbsu.edu> betz@gozer.idbsu.edu (Andrew Betz) writes:\n>In article <C5sou8.LnB@news.udel.edu> roby@chopin.udel.edu (Scott W Roby) writes:\n>>>Watch from where?  Two miles away?  Far enough away that whatever\n>>>really happenned must be explained through the vengeful filter of\n>>>a humiliated agency that said (quote!) "Enough is enough."\n>>\n>>Please tell me what you think would have happened had the people \n>>come out with their hands up several weeks ago.\n\nNo answer.\n\n>You didn\'t answer the question.  The FBI took people out of\n>camera range.  It is thus possible that they were engaging in\n>questionable activities.\n\nI do not feel like the cameras were out of range.  Cameras watched the first \nconfrontation.  Cameras watched the banners.  Cmaeras watched the final \nconfrontation with tanks.  Cameras watched the fire.  When weren\'t cameras \nable to watch?  When would cameras be unable to watch people coming out with \ntheir hands up?\n\n>As to your question, please tell me what you think would have happened\n>had the ATF goon squad knocked and asked politely several weeks\n>ago (as opposed to playing Rambo with a t.v. crew in tow).\n\nWell, that is what BATF should have done.  Either, Koresh would have gone \npeaceably as he has done in the past, or perhaps it was already too close \nto the apocalypse in his own mind.  It is hard to predict the actions of \na leader who would not release the children when most rational people would.\n\nNow will you answer my question up top?\n\n>\n>Drew\n>--\n>betz@gozer.idbsu.edu\n>*** brought into your terminal from the free state of idaho ***\n>*** when you outlaw rights, only outlaws will have rights   ***\n>*** spook fodder: fema, nsa, clinton, gore, insurrection, nsc,\n>    semtex, neptunium, terrorist, cia, mi5, mi6, kgb, deuterium\n\n\n-- \n\n\n</p>')))

In [5]:
print(train.label_distribution)

{None: 11197, False: 113, True: 4}


# 2: Live shared task

We can weakly supervise using precise functions. Note that we have no fancy model above the labelling functions.

* TODO Incorporate `snorkel` @ben

In [14]:
! mkdir -p ../submissions/username
! cp Shared\ task.ipynb ../submissions/username

Load a new dataset (`data`) from the central data pool.

In [15]:
data = Dataset.from_csv('../shared-task/data.txt')
data.add_label('foo', True)
data.add_label('bar', False)

Write back current dataset to file.

In [16]:
data.to_csv('../submissions/username/data.csv')

Contribute back to the shared task pool.

In [17]:
! git add ../submissions/username
! git commit -m 'Checkpoint username' ../submissions/username/
! git push

[master f2a6eb8] Checkpoint username
 1 file changed, 32 insertions(+)
 create mode 100644 submissions/username/Shared task.ipynb
Counting objects: 5, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 663 bytes | 663.00 KiB/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:edgedown/alta17active.git
   a1b686e..f2a6eb8  master -> master
